# Nonlinearity in Finance

> To-do list:

* Examining the definition of nonlinearity
* Discussing the volatility smile in implied volatility modeling
* Discussing Markov switching models, threshold models, and smooth transition models as nonlinear models
* An overview of root-finding to find the optimal point of nonlinear models
* Examining the incremental search algorithm, bisection algorithm, Newton's algorithm, and secant method in root-finding
* Combining root-finding methods with Brent's method
* SciPy's implementation of root-finding methods as scalar functions
* SciPy's general nonlinear solvers in root-finding


A nonlinear relationship is defined as follows:
$$ f (a+b)≠ f (a)+ f (b) $$